In [20]:
import os
import math
import wandb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import random
import numpy as np
from tensorflow.keras import datasets, layers, models
import tensorflow as tf
import pandas as pd
from collections import OrderedDict
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [2]:
(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()

x_train = x_train.reshape(60000, 28, 28, 1) / 255
x_test = x_test.reshape(10000, 28, 28, 1) / 255
x_train = np.pad(x_train, ((0, 0), (2, 2), (2, 2), (0, 0)), 'constant')
x_test = np.pad(x_test, ((0, 0), (2, 2), (2, 2), (0, 0)), 'constant')
idx = np.argsort(y_test)
x_train_sorted = x_test[idx]
y_train_sorted = y_test[idx]

UE_NUM = 20

UE = []
for _ in range(UE_NUM):
    UE.append({"x_train": [], "y_train": []})

random.seed(45)
seed = random.randint(1,1000000)
print(seed)
print(f'seed : 235511')
tf.random.set_seed(235511) # 100 좋은거, 12: 5 클러스터, 403092, 235511, 130670 : 6 클러스터, 192150 : 20

total = 0

x_eval_dataset = x_test.copy()
y_eval_dataset = y_test.copy()

data_info = []

for i in range(UE_NUM):
    random_start = random.randint(1,50000)
    #     data_num = random.randint(5000, 30000)
    data_num = random.randint(10000, 60000-random_start)
    print(random_start, random_start+data_num)
    data_info.append([random_start, random_start+data_num, random_start+data_num-random_start])
    UE[i]['x_train'] = x_train[random_start:random_start+data_num]
    UE[i]['y_train'] = y_train[random_start:random_start+data_num]

x_train, x_test, y_train, y_test = [], [], [], []
for i in range(UE_NUM):
    x_train_temp, x_test_temp, y_train_temp, y_test_temp = train_test_split(
        UE[i]['x_train'], UE[i]['y_train'], test_size=0.2, random_state=45)
    x_train.append(x_train_temp)
    x_test.append(x_test_temp)
    y_train.append(y_train_temp)
    y_test.append(y_test_temp)

285083
seed : 235511
27374 53365
16887 32253
19870 40969
1416 16156
31737 42098
7527 36129
7715 38099
48725 58798
40956 52047
8849 36647
48298 58613
13829 51903
4913 55081
26748 41427
45905 56373
2109 23395
21016 39269
23367 58765
47527 58837
6145 43106


# 1. 클러스터 성능

In [68]:
save_path = f'0930_simulation_result/2'
cluster_f1_score_list = list()

for cluster_num in range(6):
    model = tf.keras.models.load_model(
                            f'{save_path}/cluster{cluster_num}')

    predict = np.argmax(model.predict(x_eval_dataset), axis=1)
    target_names = ['0', '1', '2', '3', '4', '5', '6' ,'7' ,'8','9']
    labels = [x for x in range(10)]
    result = classification_report(y_eval_dataset, predict, labels=labels, output_dict=True)
    f1_score = list()
    for x in target_names:
        f1_score.append(result[x]['f1-score'])

    cluster_f1_score_list.append(sum(f1_score)/len(target_names))
    tf.keras.backend.clear_session()

In [70]:
sum(cluster_f1_score_list)/6

0.962261273681097

# 2. FedAvg 성능

In [71]:
save_path = f'fl_model_0930_last'
model = tf.keras.models.load_model(
                        f'{save_path}')

predict = np.argmax(model.predict(x_eval_dataset), axis=1)
target_names = ['0', '1', '2', '3', '4', '5', '6' ,'7' ,'8','9']
labels = [x for x in range(10)]
result = classification_report(y_eval_dataset, predict, labels=labels, output_dict=True)
f1_score = list()
for x in target_names:
    f1_score.append(result[x]['f1-score'])
    
tf.keras.backend.clear_session()
    
sum(f1_score)/10

0.9711787598424241